# SET Classification: Training CNN card image embedder

In [6]:
import sys

import numpy as np
import matplotlib.pyplot as plt
from setGame import *
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
sys.path.append('../..'); sys.path.append('../');
import utils

In [2]:
setgame = SetGame()

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequential
from sklearn.model_selection import train_test_split

ff_dim1=64
ff_dim2=64

img_input = layers.Input(shape=(70, 50, 4))
x = layers.Conv2D(32, (5, 5), activation='relu')(img_input)
x = layers.MaxPooling2D((4,4))(x)
x = layers.Conv2D(32, (5, 5), activation='relu')(x)
x = layers.MaxPooling2D((4,4))(x)
x = layers.Flatten()(x)
x = layers.Dense(ff_dim1, activation='relu')(x)
x = layers.Dense(ff_dim2, activation='tanh')(x)
outputs = layers.Dense(12, activation='sigmoid')(x)
model = Model(inputs=img_input, outputs=outputs)
embed = Model(model.input, model.layers[7].output)
model.summary()
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 70, 50, 4)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 66, 46, 32)        3232      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 11, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 7, 32)         25632     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 3, 1, 32)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 96)                0     

In [9]:
def convert_to_binary(attrs):
    color = {'red':[1,0,0], 'green':[0,1,0], 'purple':[0,0,1]}
    pattern = {'empty':[1,0,0], 'striped':[0,1,0], 'solid':[0,0,1]}
    shape = {'diamond':[1,0,0], 'oval':[0,1,0], 'squiggle':[0,0,1]}
    number = {'one':[1,0,0], 'two':[0,1,0], 'three':[0,0,1]}
    binary_attrs = number[attrs[0]] + color[attrs[1]] + pattern[attrs[2]] + shape[attrs[3]]
    return binary_attrs

n = 1000
X = np.empty((n, 70, 50, 4), dtype=np.float32)
y = np.empty((n, 12), dtype=int)

card_coord = [(i,j) for i in np.arange(9) for j in np.arange(9)]
for i in np.arange(n):
    c = np.random.choice(np.arange(81), size=1)[0]
    (row, col) = card_coord[c]
    attrs = setgame.attributes_of_card(row, col)
    binary_attrs = convert_to_binary(attrs)
    X[i] = setgame.image_of_card(row, col)
    y[i] = binary_attrs


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
model.fit(X_train, y_train, batch_size=128, epochs=50)

Epoch 1/50
6/6 [==============================] - 1s 22ms/step - loss: 0.6629 - binary_accuracy: 0.6292
Epoch 2/50
6/6 [==============================] - 0s 21ms/step - loss: 0.6357 - binary_accuracy: 0.6667
Epoch 3/50
6/6 [==============================] - 0s 20ms/step - loss: 0.6307 - binary_accuracy: 0.6667
Epoch 4/50
6/6 [==============================] - 0s 20ms/step - loss: 0.6221 - binary_accuracy: 0.6667
Epoch 5/50
6/6 [==============================] - 0s 21ms/step - loss: 0.6112 - binary_accuracy: 0.6690
Epoch 6/50
6/6 [==============================] - 0s 20ms/step - loss: 0.5945 - binary_accuracy: 0.6696
Epoch 7/50
6/6 [==============================] - 0s 20ms/step - loss: 0.5729 - binary_accuracy: 0.6902
Epoch 8/50
6/6 [==============================] - 0s 20ms/step - loss: 0.5500 - binary_accuracy: 0.7090
Epoch 9/50
6/6 [==============================] - 0s 21ms/step - loss: 0.5225 - binary_accuracy: 0.7331
Epoch 10/50
6/6 [==============================] - 0s 20ms/step 

In [29]:
out = model.predict(X_test, verbose=False)
pred = np.array(np.round(out), dtype=int)
print('accuracy: ', 1-np.sum(pred != y_test) / (np.prod(pred.shape)))

accuracy:  1.0


In [15]:
model.save('cnn_card_embedder/model')

INFO:tensorflow:Assets written to: cnn_card_embedder/model/assets


INFO:tensorflow:Assets written to: cnn_card_embedder/model/assets


In [16]:
embed.save('cnn_card_embedder/embedder')

INFO:tensorflow:Assets written to: cnn_card_embedder/embedder/assets


INFO:tensorflow:Assets written to: cnn_card_embedder/embedder/assets


In [26]:
card_embedder = tf.keras.models.load_model('cnn_card_embedder/embedder')